In [14]:
import numpy as np
from scipy import optimize
from scipy.optimize import Bounds, shgo, minimize
import keras
from Environment.Environment import Environment

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf



In [63]:
# Generate dist
N = 10
mean = np.zeros(2)
cov = 0.3 * np.identity(2)

dist = np.random.multivariate_normal(mean, cov, N)

In [64]:
dist

array([[-0.51279623, -0.4967897 ],
       [-0.58747512,  0.43775285],
       [ 0.06988189,  0.86023388],
       [-0.02946063,  0.05498599],
       [ 0.14451258, -0.05005101],
       [-1.46822205, -0.29139811],
       [ 0.272707  ,  0.11289018],
       [ 0.37937035, -1.04490224],
       [ 0.14055905,  0.45946203],
       [-0.01208293, -0.05910748]])

In [128]:
# Generate Network
model = keras.Sequential()
model.add(keras.layers.Input(shape=(2,)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile('adam', 'mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               300       
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 10,501
Trainable params: 10,501
Non-trainable params: 0
_________________________________________________________________


In [129]:
def f(x):
    return model.predict(x)

In [130]:
x0 = np.ones((1,2))
f(x0)

array([[0.5066248]], dtype=float32)

In [131]:
# Objective function
dim = 2
K = 1
def obj(x):
    # val = 0
    # for i in range(N):
    #     for k in range(K):
    #         start = dim*(i*N + k)
    #         stop = start + dim
    #         var = x[start:stop]
    #         val += f(var)
    # return val/(N*K)
    var = x.reshape((N*K,dim))
    return -np.sum(f(var), axis=0)[0]/(N*K)

In [132]:
x = np.random.rand(N*K*dim)
obj(x)

-0.5031237602233887

In [133]:
theta = 10.0
def constraint(x):
    dist_k = np.repeat(dist, repeats=K, axis=0)
    x_m = x.reshape(N*K,dim)
    val = np.square(np.linalg.norm(dist_k-x_m,2,axis=1))
    return np.sum(val)/(N*K)

constraint(x)
    

1.3804066486285673

In [134]:
def jac(x):
    dist_k = np.repeat(dist, repeats=K, axis=0).flatten()
    return 2*(x - dist_k)/(N*K)

In [135]:
def df(x):
    inp = x.reshape((-1,dim))
    xc = tf.Variable(inp, dtype=tf.float32)
    with tf.GradientTape() as g:
        g.watch(xc)
        pred = model(xc)
        grad = g.jacobian(pred,xc)
        # print(grad)
    return grad.numpy()

np.array([df(x0),df(x0)]).flatten()

array([-0.01186966,  0.01849368, -0.01186966,  0.01849368], dtype=float32)

In [136]:
#a = np.array([np.ones(2), 2*np.ones(2)])
a = np.ones((10,2))
xc = tf.Variable(a, dtype=tf.float32)
with tf.GradientTape() as g:
    g.watch(xc)
    pred = model(xc)
    grad = g.jacobian(pred,xc)

print(grad.numpy().shape)

j_sum = tf.reduce_sum(grad.numpy(), axis=2)
print(j_sum.shape)
print(j_sum.numpy().flatten())

(10, 1, 10, 2)
(10, 1, 2)
[-0.01186966  0.01849367 -0.01186966  0.01849367 -0.01186966  0.01849367
 -0.01186966  0.01849367 -0.01186966  0.01849367 -0.01186966  0.01849367
 -0.01186966  0.01849367 -0.01186966  0.01849367 -0.01186966  0.01849367
 -0.01186966  0.01849367]


In [137]:
def jac_f(x):
    # val = []
    # for i in range(N):
    #     for k in range(K):
    #         start = dim*(i*K + k)
    #         stop = start + dim
    #         # print(f"{start} : {stop}")
    #         var = x[start:stop]
    #         # print(var)
    #         val.append(df(var))
    # return -np.array(val).flatten()
    
    x_m = x.reshape((-1,dim))
    j = df(x)
    j_sum = tf.reduce_sum(j, axis=2)
    return j_sum.numpy().flatten()

jac_f(np.ones((3,2)))

array([-0.01186966,  0.01849368, -0.01186966,  0.01849368, -0.01186966,
        0.01849368], dtype=float32)

In [138]:
from scipy.optimize import NonlinearConstraint

con = NonlinearConstraint(constraint, -np.inf, theta**2)

In [139]:
lb = -10*np.ones(N*K*dim)
ub = 10*np.ones(N*K*dim)
bounds = Bounds(lb,ub)
options = {'maxiter':200}
result = minimize(obj, x, method='SLSQP', bounds=bounds, jac=jac_f, constraints=con, options=options)
print(-obj(result.x))
print(result)

0.503123664855957
     fun: -0.503123664855957
     jac: array([-0.0135251 ,  0.02240705,  0.00610416,  0.00296268,  0.00284762,
        0.00332205, -0.01187076,  0.0184954 ,  0.00109883,  0.0048399 ,
        0.00322664,  0.00291491, -0.01270182,  0.02045813, -0.00507491,
        0.01160968, -0.00927379,  0.01554166,  0.00257245,  0.0043512 ])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([0.48964063, 0.29777103, 0.66119083, 0.94309548, 0.5321398 ,
       0.49106492, 0.67484306, 0.67925904, 0.04316   , 0.12899232,
       0.37535634, 0.35377668, 0.77434423, 0.63854256, 0.77276655,
       0.75634392, 0.62662272, 0.42661353, 0.27925514, 0.94626946])


In [ ]:
ineq_cons = {'type': 'ineq',
             'fun': constraint,
             'jac': jac}
bounds = []
for i in range(N*K*dim):
    bounds.append((-10,10))
options = {'maxiter':1, 'disp':True}
results = optimize.shgo(obj, bounds, options=options)

In [10]:
a = np.array([[1,4],[2,5],[3,6]])
np.repeat(a, repeats=1, axis=0)

array([[1, 4],
       [2, 5],
       [3, 6]])

In [49]:
b = a.flatten()
b.reshape((3,2))


array([[1, 4],
       [2, 5],
       [3, 6]])

In [54]:
np.linalg.norm(a - b.reshape((3,2)),2,axis=1)

array([0., 0., 0.])